In [ ]:
# pip install requests pandas tqdm tenacity

In [1]:
from catalogo_teses import datasets

df = datasets.get_all_datasets_with_resources()
df = datasets.get_dataset_by_name(df)

In [2]:
from tqdm.auto import tqdm
from typing import Iterable

def download_files(urls: Iterable[str], dest_dir: str):
    """Download multiple files to the given directory."""
    files = []
    for url in tqdm(urls, desc="Baixando..."):
        files.append(datasets.download_file(url, dest_dir))
    return files


In [3]:
download_files([df['url'].iloc[-1]], "./data")

Baixando...:   0%|          | 0/1 [00:00<?, ?it/s]

dados_2012.csv:   0%|          | 0/156 [00:00<?, ?it/s]

['./data/dados_2012.csv']